In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
def find_third_largest(nums):
    if len(nums) < 3:
        return "列表长度不足3"
    
    # 将列表去重并排序
    unique_nums = sorted(set(nums), reverse=True)
    
    if len(unique_nums) < 3:
        return "列表中没有第三大的数字"
    
    return unique_nums[2]

In [3]:
RAData = pd.read_csv("RA medication class(filtered).csv")
RAData = RAData.drop_duplicates(subset=["MedicationCui","ClassCui"]).reset_index(drop=True)

In [4]:
## Change T2DData[['MedicationStr','ClassStr']] into Dictionary. The key is ClassStr, and the value is MedicationStr which belongs to the ClassStr.
RADataDict = {}
for i in range(len(RAData)):
    if RAData.loc[i,'ClassStr'] not in RADataDict:
        RADataDict[RAData.loc[i,'ClassStr']] = [RAData.loc[i,'MedicationStr']]
    else:
        RADataDict[RAData.loc[i,'ClassStr']].append(RAData.loc[i,'MedicationStr'])

In [5]:
## Intersection Number
IntersectionNumber = {}
for key1 in RADataDict.keys():
    IntersectionNumber[key1] = 0
    for key2 in RADataDict.keys():
        if key1 != key2 and len(set(RADataDict[key1]) & set(RADataDict[key2]))>0:
            IntersectionNumber[key1] += 1

In [6]:
## find the class which has the most intersection with other classes
class_to_remove = []

threshold = find_third_largest(list(IntersectionNumber.values()))
for key in IntersectionNumber.keys():
    if IntersectionNumber[key] >= threshold:
        class_to_remove.append(key)
        
for key in class_to_remove:
    del RADataDict[key]

In [7]:
## remove the key which is the subset of other key
class_to_remove = []  
class_used = []
for key in RADataDict.keys():
    data = RADataDict[key]
    for supkey in RADataDict.keys():
        if supkey == key or supkey in class_used:
            continue
        supdata = RADataDict[supkey]
        if set(data).issubset(set(supdata)):
            class_used.append(key)
            class_to_remove.append(key)  

class_to_remove = list(set(class_to_remove))

for key in class_to_remove:
    del RADataDict[key]


In [8]:
#Judge the number of the class contain one medication
single_class = []
for key in RADataDict.keys():
    if len(RADataDict[key]) == 1:
        single_class.append(key)
#Change the key of the single class to 'other' 
if len(single_class) > 0:
    RADataDict['other'] = RADataDict[single_class[0]]
    del RADataDict[single_class[0]]
    for i in range(1,len(single_class)):
        RADataDict['other'] = RADataDict['other'] + RADataDict[single_class[i]]
        del RADataDict[single_class[i]]

In [9]:
#change the dictionary into dataframe
RAData = pd.DataFrame(columns=['ClassStr', 'MedicationStr'])
for key in RADataDict.keys():
    for value in RADataDict[key]:
        row = {'ClassStr': key, 'MedicationStr': value}
        RAData = pd.concat([RAData, pd.DataFrame([row])])

In [10]:
RAData['Disease'] = 'RA'

In [11]:
RAData.to_csv("PlotMedicationData(RA).csv",index=False,encoding='utf_8_sig')

## ALL

In [11]:
fig = px.sunburst(RAData,
                  path=["Disease", "ClassStr", "MedicationStr",],
                  width=750, height=750,
                  title="RA Medication(Rule Filtered)",
                  )
fig.write_html("RA Medication(Rule Filtered).html")
fig.show()

## Ori